<h1 style="width:750px">Airbnb and the Prevalence of Ghost Hotels in London</h1>

---

| CASA0013 | Introduction to Programming for Spatial Analysts | 
|----|----|
| Programme | MSc Smart Cities and Urban Analytics |
| Bartlett | Centre for Advanced Spatial Analytics |
| Word Count | 1,647 |
| Student ID | 19140941 |
| Date | 22-02-09 |

---

## Part 1 - Reproducible Analysis

Data for this analysis was downloaded from the Inside Airbnb website at http://insideairbnb.com/get-the-data.html.

| Data Downloaded | |  | |
|----|----|----|----|
|  09 August, 2019  |  London  |  listings.csv.gz  |  Detailed Listings data for London  |
|  N/A  |  London  |  neighbourhoods.csv  |  Neighbourhood list for geo filtering  |
|  N/A  |  London  |  neighbourhoods.geojson  |  GeoJSON file of city neighbourhoods  |

**Code:**

The csv file used in the reproducible analysis below and associated outputs can be downloaded from [GitHub](https://github.com/MattKavanaugh/I2P-Exec-Briefing). 

### Load Data

In [ ]:
# load libraries
import csv
import os
from urllib.request import urlopen
from IPython import display

# Load numeric libraries
import geopandas as gpd
import math
%matplotlib inline
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import sklearn
        
# load spatial libraries
import json
import pysal as ps
import mapclassify as mc
import palettable.matplotlib as palmpl
from legendgram import legendgram

In [ ]:
# open data into a pandas dataframe
url = 'https://github.com/MattKavanaugh/i2p_a3_final/blob/main/19-08_listings.csv.gz?raw=true'
pdf = pd.read_csv(url, compression='gzip', low_memory=False)

# for local use only
# pdf = pd.read_csv('19-08_listings.csv.gz', compression='gzip', low_memory=False)

# confirm load
print(f"Data frame is {pdf.shape[0]:,} columns x {pdf.shape[1]} rows")

### Review & Reduce Data

In [ ]:
# check dataframe
pdf.info()

In [ ]:
# check column names
print(list(pdf.columns.values))

In [ ]:
# look at data
pdf.head()

In [ ]:
# drop columns we don't think we will need or that we think have any directly identifiable information
cols2drop = ['listing_url', 'scrape_id', 'last_scraped', 'summary', 'space', 'experiences_offered', 'notes', 
             'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 
             'xl_picture_url', 'host_url', 'host_name', 'host_since', 'host_about', 'host_response_time', 'host_response_rate', 
             'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 
             'host_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified', 
             'neighbourhood', 'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market', 
             'smart_location', 'country_code', 'country', 'is_location_exact', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 
             'amenities', 'square_feet', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 
             'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
             'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 
             'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365', 
             'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm', 'first_review', 'last_review', 
             'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
             'review_scores_communication', 'review_scores_location', 'review_scores_value', 'requires_license', 
             'license', 'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready', 'cancellation_policy', 
             'require_guest_profile_picture', 'require_guest_phone_verification', 'calculated_host_listings_count_shared_rooms', 
             'reviews_per_month']
for col in pdf.columns:
    if col in cols2drop:
        pdf.drop(col, inplace=True, axis=1)

In [ ]:
# check reduced dataframe
print(f"Data frame is {pdf.shape[0]:,} columns x {pdf.shape[1]} rows")

# check reduced column names
print(list(pdf.columns.values))

In [ ]:
# look at a few columns in more detail to see if we can drop more
pdf.head()[['name', 'description', 'neighborhood_overview', 'host_location', 
           'host_neighbourhood', 'host_total_listings_count', 'street', 
           'neighbourhood_cleansed', 'price', 'weekly_price', 'monthly_price', 
           'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
           'calculated_host_listings_count_private_rooms']]

In [ ]:
# Data review
# 1. Descriptions - 3 fields
    # the name, description, and neighoburhood overview fields seem to come directly from the hosts
    # these will be valuable for later analysis

# 2. Locations - 4 fields
    # the host_location, host_neighbourhood, street, and neighbourhood cleansed fields are all over the place
    # likely partly to user error during entry (deliberate or otherwise), scraping methods, and/or Airbnb itself
    # for ease of use, we will keep the neighbourhood_cleansed and drop the other 3

# 3. Host Listing Counts - 4 fields
    # the host__listings_count fields also seem to vary and can easily be calculated on our own
    # we will drop all 4 of these
    
# 4. Prices - 3 fields
    # we will need to convert these to remove the $ sign

morecols2drop = ['host_location', 'host_neighbourhood', 'host_total_listings_count', 'street', 
                 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                 'calculated_host_listings_count_private_rooms']
for col in pdf.columns:
    if col in morecols2drop:
        pdf.drop(col, inplace=True, axis=1)

In [ ]:
# check final dataframe
print(f"Data frame is {pdf.shape[0]:,} columns x {pdf.shape[1]} rows")

# check final column names
print(list(pdf.columns.values))

In [ ]:
# save raw data
# ADD THIS WHEN FROM GITHUB - fn   = url.split('/')[-1]
path = os.path.join('data','raw')
fn = '19-08_listings'
print(f"Writing to: {fn}")

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
pdf.to_csv(os.path.join(path,fn), index=False)
print("Done.")

### Clean Data

In [ ]:
# reload data
del(pdf)
pdf = pd.read_csv(os.path.join(path,fn), low_memory=False)
print(f"Data frame is {pdf.shape[0]:,} columns x {pdf.shape[1]} rows")

In [ ]:
# first we will count nulls by columns to identify if there are any problems
pdf.isnull().sum(axis=0).sort_values(ascending=False)

In [ ]:
# we will drop 'monthly_price','weekly_price', and 'neighborhood_overview'
pdf.drop(columns=['neighborhood_overview', 'weekly_price', 'monthly_price'], inplace=True)

In [ ]:
# we will now count nulls by rows
pdf.isnull().sum(axis=1).sort_values(ascending=False)

In [ ]:
# no need to drop any

# now let's see how many rows have NaNs in key columns
pdf[pdf.id.isna()][['id', 'name', 'host_id']]

In [ ]:
# no issues

# now let's check datatypes
pdf.dtypes

In [ ]:
# Data to clean

# 1. The following fields will need to be converted to integer
    # 'id', 'host_id', 'accommodates', 'minimum_nights'

# 2. The following fields will need to be converted to float
    # 'latitude', 'longitude', 'price'
    
# 3. The following fields will need to be converted to string
    # 'name', 'description'

# 4. A geometry column will need to be created from the 'latitude' and 'longitude' fields

##### Convert selected columns to integer

In [ ]:
# convert 'id', 'host_id', 'accommodates', 'minimum_nights'
pdf['id'] = pdf['id'].astype("int")
pdf['host_id'] = pdf['host_id'].astype("int")
pdf['accommodates'] = pdf['accommodates'].astype("int")
pdf['minimum_nights'] = pdf['minimum_nights'].astype("int")

# confirm changes
pdf.dtypes

##### Convert selected columns to float

In [ ]:
# convert price
pdf['price'] = pdf['price'].str.replace("[$, ]", "").astype("float")

# convert latitude and longitude
pdf['latitude'] = pdf['latitude'].astype("float")
pdf['longitude'] = pdf['longitude'].astype("float")

# confirm changes
pdf.dtypes

##### Reduce to only 'short term rentals'

In [ ]:
# monthly sublets are a legitimate practice that can be found throughout cities worldwide, especially those with student populations
# to avoid including any of these we will classify a short term rental as any listing offering
# a minimum night stay less than 28 days (to account for the month of February)
# the rest we will drop

# let's see how many listings we have now
pdf['id'].count()

In [ ]:
# we will create indexes of listings where minimum_nights is greater than 28
indexNames = pdf[pdf['minimum_nights'] > 28].index

# Drop these listings
pdf.drop(indexNames , inplace=True)

# let's see how many listings we have left
pdf['id'].count()

In [ ]:
tl = 85918
stl = 84412

print(f"{round(((stl/tl)*100), 2)}% of all listings offer short term rentals")

In [ ]:
# we no longer need the minimum nights column so it can be dropped
pdf.drop('minimum_nights', axis=1, inplace=True)

# confirm changes
pdf.info()

##### Clean up categories

In [ ]:
# check property type counts by percentage
pdf.property_type.value_counts(normalize=True)

In [ ]:
# The top five seem to represent the vast majority of all listings
pdf.property_type.value_counts(normalize=True)[:5].sum().round(2)

In [ ]:
# we will keep the top 5 and merge all else into an "Other" label

top5 = list(pdf.property_type.value_counts(normalize=True)[:5].index)

pdf.property_type = [x if x in top5 else 'Other' for x in pdf.property_type]

pdf.property_type.value_counts(normalize=True)

##### Identify and remove outliers

In [ ]:
# use the Interquartile Rule (IQR Score)
    # Calculate the IQR
    # Multiply the IQR by 1.5 (a constant used to discern outliers)
    # Add 1.5 x (IQR) to the third quartile (75%). Any number greater than this is a suspected outlier.
    # Subtract 1.5 x (IQR) from the first quartile (25%). Any number less than this is a suspected outlier.
# credit to: https://github.com/brunacmendes/Airbnb_Data_Analysis_Toronto/blob/main/Airbnb_Listings_Data_Analysis_Toronto.ipynb
    
# first select relevant data
pdf[['accommodates', 'price']].describe().round(1)

In [ ]:
# Displaying the box plot for accommodates
fig, ax = plt.subplots(figsize=(15,3))
pdf['accommodates'].plot(kind='box', vert=False)
ax.set_title('Original dataset - accommodates')
plt.show()

# IQR rule for variable accommodates
q1_mn = pdf.accommodates.quantile(q=0.25)
q3_mn = pdf.accommodates.quantile(q=0.75)
IQR_mn = q3_mn - q1_mn

print('IQR accommodates: ', IQR_mn)

sup_mn = q3_mn + 1.5 * IQR_mn
inf_mn = q1_mn - 1.5 * IQR_mn

print('Upper limit: ', sup_mn)
print('Lower limit: ', inf_mn)

print("{} Entries above upper limit".format(len(pdf[pdf.accommodates > sup_mn])))
print("{:.2f}% of dataset".format((len(pdf[pdf.accommodates > sup_mn]) / pdf.shape[0])*100))

In [ ]:
# Display the box plot for price
fig, ax = plt.subplots(figsize=(15,3))
pdf['price'].plot(kind='box', vert=False)
ax.set_title('Original dataset - price')
plt.show()

# IQR rule for variable price
q1_price = pdf.price.quantile(q=0.25)
q3_price = pdf.price.quantile(q=0.75)
IQR_price = q3_price - q1_price

print('IQR price: ', IQR_price)

sup_price = q3_price + 1.5 * IQR_price
inf_price = q1_price - 1.5 * IQR_price

print('Upper limit: ', sup_price)
print('Lower limit: ', inf_price)

print("{} Entries above upper limit".format(len(pdf[pdf.price > sup_price])))
print("{:.2f}% of dataset".format((len(pdf[pdf.price > sup_price]) / pdf.shape[0])*100))

In [ ]:
# Select the rows that fulfil the established requirements
pdf = pdf[(pdf['price'] <= 282.5) & (pdf['price'] > 0) & (pdf['accommodates'] <= 7.0)].copy()

#plotting the boxplots again
fig, ax = plt.subplots(figsize=(15,3))
pdf['price'].plot(kind='box', vert=False)
ax.set_title('Clean dataset - price')
plt.show()

fig, ax = plt.subplots(figsize=(15,3))
pdf['accommodates'].plot(kind='box', vert=False)
ax.set_title('Clean dataset - accommodates')
plt.show()

In [ ]:
# confirm changes 
pdf[['accommodates', 'price']].describe().round(1)

##### Add geometry

In [ ]:
# create a geometry column and geodataframe from the 'latitude' and 'longitude'
gdf = gpd.GeoDataFrame(pdf, 
      geometry=gpd.points_from_xy(pdf['longitude'], pdf['latitude'], crs='epsg:4326'))

In [ ]:
# confirm created
print(gdf.geometry.crs)
print(gdf.total_bounds)
gdf.plot(figsize=(12,8), marker='*', color='green', markersize=0.25)

In [ ]:
# change projection to BNG
gdf = gdf.to_crs("epsg:27700")
print(gdf.geometry.crs)
print(gdf.total_bounds)
gdf.plot(figsize=(12,8), marker='*', color='green', markersize=0.5)

In [ ]:
# read in json file
lon_nbs = gpd.read_file('http://data.insideairbnb.com/united-kingdom/england/london/2021-12-07/visualisations/neighbourhoods.geojson')
lon_nbs.head()

In [ ]:
# we will need to merge neighbourhood with neighbourhood_cleansed from our dataframe
# we will rename neighbourhood and drop neighbourhood group to avoid confusion
lon_nbs.rename(columns={'neighbourhood': 'neighbourhood_cleansed'}, inplace=True)
lon_nbs.drop('neighbourhood_group', axis=1, inplace=True)

# check to confirm
lon_nbs.head()

In [ ]:
# plot 
print(lon_nbs.geometry.crs)
print(lon_nbs.total_bounds)
ax = lon_nbs.plot(figsize=(10,7), edgecolor='red', facecolor='none', linewidth=1, alpha=0.75)

In [ ]:
# change projection to BNG
lon_nbs = lon_nbs.to_crs("epsg:27700")
print(gdf.geometry.crs)
print(gdf.total_bounds)
ax = lon_nbs.plot(figsize=(10,7), edgecolor='red', facecolor='none', linewidth=1, alpha=0.75)

### Save Data

In [ ]:
# final check of dataframe
pdf.head()

In [ ]:
# save dataframe

path = os.path.join('data','clean')
fn = '19-08_listings'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
pdf.to_csv(os.path.join(path,fn), index=False)
print("Done.")

### Save Geodata

In [ ]:
# check geodataframe
gdf.head()

In [ ]:
# save geodataframe
gdf.to_file('19-08-listings.gpkg', driver='GPKG')

# Set save location
path = os.path.join('data','geo')
fn = '19-08-listings.gpkg'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
print(f"Using '{fn}' as basis for saving data...")

gdf.to_file(os.path.join('data','geo',fn.replace('.csv.gz','.gpkg')), driver='GPKG')
print("Done.")

#### Analysis

In [ ]:
# let's see the statistics for all short term rentals

print(f"in our sample of all short term listings, there are {gdf.id.nunique()} total listings however there are only {gdf.host_id.nunique()} total hosts ...")
print(f"... meaning there are {(gdf.id.nunique())-(gdf.host_id.nunique())} or {round((25571/76692)*100,2)}% more listings than hosts...")
print(f"... with enough nightly accommodations for {pdf.accommodates.sum()} people with an average of {round(gdf.accommodates.mean(),1)} people per listing ...")
print(f"... an average price for a listing of ${round(gdf.price.mean(),2)} per night...")
print(f"... and at a profit of ${round(gdf.price.sum())} per night if all listings were booked at the same time")

In [ ]:
# now lets check statistics for all short term listings by hosts with more than one listing
multi_host = gdf[gdf.host_id.duplicated(keep=False)]

In [ ]:
# let's see the stats for all short term multi-host listings
print(f"in our sample of short term multi-host listings, there are {multi_host.id.nunique()} total listings however there are only {multi_host.host_id.nunique()} total hosts ...")
print(f"... meaning there are {(multi_host.id.nunique())-(multi_host.host_id.nunique())} or {round((9186/34757)*100,2)}% more listings than hosts ...")
print(f"... with enough nightly accommodations for {multi_host.accommodates.sum()} people with an average of {round(multi_host.accommodates.mean(),1)} people per listing ...")
print(f"... an average price for a listing of ${round(multi_host.price.mean(),2)} per night ...")
print(f"... and at a profit of ${round(multi_host.price.sum())} per night if all listings were booked at the same time.")

In [ ]:
# and lets compare the stats
print(f"in comparison, multi-host listings make up {round(multi_host.id.nunique()/gdf.id.nunique()*100,2)}% of total listings...")
print(f"... meaning that only {multi_host.host_id.nunique()} or {round(multi_host.host_id.nunique()/gdf.host_id.nunique()*100,2)}% of all hosts control ...")
print(f"... {round(multi_host.id.nunique()/gdf.id.nunique()*100,2)}% of all listings ...")
print(f"... {round(multi_host.accommodates.sum()/gdf.accommodates.sum()*100,2)}% of all nightly accommodations at an average of {round(((multi_host.accommodates.mean())-(gdf.accommodates.mean())),1)} more people per listing ...")
print(f"... and at an average of ${round(((multi_host.price.mean())-(gdf.price.mean())),2)} more per night ...")
print(f"... for approximately {round(multi_host.price.sum()/gdf.price.sum()*100,2)}% of nighlty profits for all listings, if they were all booked at the same time.")

In [ ]:
# visualize statistics
plt.figure(figsize = [10, 8])

# Define data coordinates and labels
hosts = ['41,935\nsingle hosts', '9,186\nmutli-hosts']
hosts_num = [41935, 9186]
listings = ['41,935\nsingle-hosts', '34,757\nmutli-hosts']
listing_num = [41935, 34757]
accoms = ['113,005 by\nsingle-hosts', '104,487 by\nmutli-hosts']
accoms_num = [113005, 104487]
profit = ['$3.5 mil from\nsingle-hosts', '$3.3 mil from\nmutli-hosts']
profit_num = [3469210, 3332875]

# number of hosts 
plt.subplot(2, 2, 1)
plt.pie(hosts_num,labels=hosts, colors = ['darkgoldenrod','gold'], autopct='%1.1f%%', startangle=70, wedgeprops={'alpha':0.9})
plt.title('Hosts', fontsize=14, color='black')

# number of listings
plt.subplot(2, 2, 2)
plt.pie(listing_num,labels=listings, colors = ['darkgoldenrod','gold'], autopct='%1.1f%%', startangle=120, wedgeprops={'alpha':0.9})
plt.title('Listings', fontsize=14, color='black')

# number of people being accomodated
plt.subplot(2, 2, 3) 
plt.pie(accoms_num,labels=accoms, colors = ['darkgoldenrod','gold'], autopct='%1.1f%%', startangle=120, wedgeprops={'alpha':0.9})
plt.title('Guests per night', fontsize=14, color='black')

# amount of profit
plt.subplot(2, 2, 4) 
plt.pie(profit_num,labels=profit, colors = ['darkgoldenrod','gold'], autopct='%1.1f%%', startangle=120, wedgeprops={'alpha':0.9})
plt.title('Profit per night', fontsize=14, color='black')

# save fig
path = os.path.join('','img')
fn = 'fig1'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

# show plot
plt.show()

In [ ]:
# number of multi host listings by type of property
multi_host.property_type.value_counts()

In [ ]:
# apartments make up more multi host listings than all other categories combined
# let's look at number of multi host listings by type of room
multi_host.room_type.value_counts()

In [ ]:
# and the percentages ..
# percentage of multi host listings by room type
multi_host_room_per = round((multi_host.room_type.value_counts() / multi_host.shape[0] *100), 1)
print(multi_host_room_per)

In [ ]:
# theres a more even split here between entire homes/aprts and private rooms
# let's visualize this ...
plt.figure(figsize = [12, 8])

# first subplot (1 row, 2 columns, position 1)
plt.subplot(1, 2, 1)
multi_host_freq = multi_host['room_type'].value_counts().sort_values(ascending=True)
multi_host_freq.plot.barh(figsize =(10,3), width=1, color=['khaki','gold','goldenrod','darkgoldenrod'])
plt.title("Multi-host Listings by Room Type", fontsize=14)
plt.grid(True)

# second subplot (1 row, 2 columns, position 2)
plt.subplot(1, 2, 2)
labels = ['18575\nHomes & Apt', '15844\nPrivate', '338\nShared & Hotel']
sizes = [53.4, 45.6, 1.0]
plt.pie(sizes,labels=labels, colors = ['darkgoldenrod','goldenrod','gold'], autopct='%1.1f%%', startangle=120, wedgeprops={'alpha':0.8})
plt.title('Multi-host Listings by Room Type', fontsize=12, color='black', fontweight='bold')

# save fig
path = os.path.join('','img')
fn = 'fig2'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

plt.show()

In [ ]:
# check counts
multi_host.room_type.value_counts()

In [ ]:
# check total
multi_host.id.count()

In [ ]:
# average price for multi listings by neighbourhood

prices = multi_host.groupby(
    ["neighbourhood_cleansed"], as_index=False
).agg(
    avg_price = ("price", "mean"),
    qty = ("price", "count")
).sort_values(
    by="avg_price", ascending=False
).reset_index(drop=True)

print(prices)

In [ ]:
prices_map = lon_nbs.merge(prices, on='neighbourhood_cleansed')
prices_map.head()

In [ ]:
ax = prices_map.plot(figsize=(12,9), edgecolor='black', linewidth=1, alpha=0.7, column='avg_price', cmap='copper', scheme='Fisher_Jenks', legend=True, k=10)
#ax = multi_host.plot(markersize=0.1, color = 'black', scheme='Fisher_Jenks', ax=ax)
ax.set_title("Mean Price Distribution by Borough", fontsize=14)
plt.axis('off')

path = os.path.join('','img')
fn = 'fig3'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

In [ ]:
# distribution of multi-listers by neighbourhood
ax = prices_map.plot(figsize=(12,9), edgecolor='black', linewidth=1, alpha=0.7, column='qty', cmap='cividis', scheme='Fisher_Jenks', legend=True, k=8)
#ax = multi_host.plot(markersize=0.1, color = 'black', scheme='Fisher_Jenks', ax=ax)
ax.set_title("Distribution of Multi-host Listings by Borough", fontsize=14)
plt.axis('off')

path = os.path.join('','img')
fn = 'fig4'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

In [ ]:
# geographic distribution of multi lisitings
multi_host['neighbourhood_cleansed'].value_counts().sort_values(ascending=False)

In [ ]:
# only keep neighbourhoods with over 600 listings (top 16)
multi_host['neighbourhood_cleansed'].value_counts().loc[lambda x : x>600] 

In [ ]:
# distribution of multi-listers by neighbourhood
nbd_distrib = multi_host['neighbourhood_cleansed'].value_counts().loc[lambda x : x>600]
nbd_distrib.plot.barh(figsize =(12,8), width=1, color=['darkgoldenrod','gold'], edgecolor = "black")
plt.title("Boroughs with over 600 Multi-host Listings", fontsize=14)
plt.grid(True)

# save fig
path = os.path.join('','img')
fn = 'fig5'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

plt.show()

In [ ]:
# create a subset for our search for ghost hotels
# we will drop shared rooms and hotel rooms right away and leave just apartments and private rooms
ghost_hunt = gdf[gdf.host_id.duplicated(keep=False)]
ghost_hunt = ghost_hunt[ghost_hunt.room_type != 'Shared room']
ghost_hunt = ghost_hunt[ghost_hunt.room_type != 'Hotel room']

In [ ]:
ghost_hunt.id.count()

In [ ]:
# hosts with more than 25 listings
freq1 = ghost_hunt.groupby(['host_id']).size().reset_index(name='num_host_listings')
freq1 = freq1.sort_values(by=['num_host_listings'], ascending=False)
freq1 = freq1[freq1['num_host_listings'] >= 25]
freq1

In [ ]:
freq1.sum()

In [ ]:
freq1.count()

In [ ]:
# hosts with more than 50 listings
freq2 = ghost_hunt.groupby(['host_id']).size().reset_index(name='num_host_listings')
freq2 = freq2.sort_values(by=['num_host_listings'], ascending=False)
freq2 = freq2[freq2['num_host_listings'] >= 50]
freq2

In [ ]:
# hosts with more than 75 listings
#Display hosts with more than 75 listings
freq3 = ghost_hunt.groupby(['host_id']).size().reset_index(name='num_host_listings')
freq3 = freq3.sort_values(by=['num_host_listings'], ascending=False)
freq3 = freq3[freq3['num_host_listings'] >= 75]
freq3

In [ ]:
# hosts with more than 100 listings
#Display hosts with more than 100 listings
freq4 = ghost_hunt.groupby(['host_id']).size().reset_index(name='num_host_listings')
freq4 = freq4.sort_values(by=['num_host_listings'], ascending=False)
freq4 = freq4[freq4['num_host_listings'] >= 100]
freq4

In [ ]:
# let's look at the top host with 646 listings
sus1 = ghost_hunt[ghost_hunt['host_id']==33889201]
sus1 = sus1[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus1.index.name = "id"
sus1.sample(8, random_state=42)

In [ ]:
# the listings geographically scattered, but the listing name gives it away - "Veeve"
sus1['name']

In [ ]:
# a quick search reveals that Veeve actually seems to be either a competitor or partner to Airbnb
# https://www.veeve.com/en-gb/
# this is a proffesionally managed host

# let's try another - host 21867370 with 51
sus2 = ghost_hunt[ghost_hunt['host_id']==21867370]
sus2 = sus2[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus2.index.name = "id"
sus2.sample(8, random_state=42)

In [ ]:
# Listing geographies again seem scattered, but the description and names are to identical and have been classified by the host
# seems to be another professionally managed host

# lets try another - this time host 57166466 with 31
sus3 = ghost_hunt[ghost_hunt['host_id']==57166466]
sus3 = sus3[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus3.index.name = "id"
sus3.sample(8, random_state=42)

In [ ]:
# same trend here - the posts are nearlly identical and have been classified by the host
# will call this also professionally managed

# last one - this time host 40412482 with 25
sus4 = ghost_hunt[ghost_hunt['host_id']==40412482]
sus4 = sus4[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus4.index.name = "id"
sus4.sample(8, random_state=42)

In [ ]:
# once again, this hosts listings are identical and seperated by a code

# it is clear that property management companies are having a major impact
# it seems like host ids are seemingly more likely to correspond to an agent then an acutal host
# this means that the actual property owner could literally be anyone (investor, business/corporation, indidvudal, foregin property owner)

In [ ]:
# lets do the same excersie but with just private rooms this time
private_rooms = ghost_hunt[ghost_hunt['room_type'] == "Private room"]
pr_hosts = private_rooms.groupby(['host_id']).size().reset_index(name='private_rooms').sort_values(by=['private_rooms'], ascending=False)
pr_hosts.head(60)

In [ ]:
# lets look at four randomly across the range

# first the top host 13557337 with 112
sus5 = ghost_hunt[ghost_hunt['host_id']==13557337]
sus5 = sus5[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus5.index.name = "id"
sus5.sample(8, random_state=42)

In [ ]:
# identical listing names and descriptiions, seperated by code, same neighbourhood

# now host 56784981 with 39
sus6 = ghost_hunt[ghost_hunt['host_id']==56784981]
sus6 = sus6[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus6.index.name = "id"
sus6.sample(8, random_state=42)

In [ ]:
# another host with identical listing names and descriptiions, seperated by code, and in the same neighbourhood

# now host 51439158 with 23
sus7 = ghost_hunt[ghost_hunt['host_id']==51439158]
sus7 = sus7[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus7.index.name = "id"
sus7.sample(8, random_state=42)

In [ ]:
# similar listing names and descriptiions, but more geographically dispursed

# lastly host 17539274 with 15
sus8 = ghost_hunt[ghost_hunt['host_id']==17539274]
sus8 = sus8[['id', 'name', 'host_id', 'description', 'neighbourhood_cleansed', 'property_type', 'room_type', 'geometry']]
sus8.index.name = "id"
sus8.sample(8, random_state=42)

In [ ]:
# this host has codes again, similar descriptions, and seemingly favoured neighbourhoods

# lets plot our suspects to get a sense of their geographic concentration

sus1 = ghost_hunt[ghost_hunt['host_id']==33889201].copy()
print(sus1.shape)

sus2 = ghost_hunt[ghost_hunt['host_id']==21867370].copy()
print(sus2.shape)

sus3 = ghost_hunt[ghost_hunt['host_id']==57166466].copy()
print(sus3.shape)

sus4 = ghost_hunt[ghost_hunt['host_id']==40412482].copy()
print(sus4.shape)

sus5 = ghost_hunt[ghost_hunt['host_id']==13557337].copy()
print(sus5.shape)

sus6 = ghost_hunt[ghost_hunt['host_id']==56784981].copy()
print(sus6.shape)

sus7 = ghost_hunt[ghost_hunt['host_id']==51439158].copy()
print(sus7.shape)

sus8 = ghost_hunt[ghost_hunt['host_id']==17539274].copy()
print(sus8.shape)

In [ ]:
plt.figure(figsize = [12, 8])

ax = lon_nbs.plot(figsize=(12,9), edgecolor='gray', facecolor='none', linewidth=1, alpha=0.9, scheme='Fisher_Jenks')

# Host 1 - 646
ax1 = sus1.plot(markersize=8, color = 'lightskyblue', scheme='Fisher_Jenks', ax=ax)
# Host 2 - 51
ax2 = sus2.plot(markersize=10, color = 'darkgreen', scheme='Fisher_Jenks', ax=ax)
# Host 3 - 31
ax3 = sus3.plot(markersize=10, color = 'darkorange', scheme='Fisher_Jenks', ax=ax)
# Host 4 - 25
ax4 = sus4.plot(markersize=10, color = 'darkred', scheme='Fisher_Jenks', ax=ax)

ax.set_title("Geographic Distribution of Sample One\n\n4 hosts / 752 listings", fontsize=14)
plt.axis('off');

path = os.path.join('','img')
fn = 'fig6'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

In [ ]:
plt.figure(figsize = [12, 8])

ax = lon_nbs.plot(figsize=(12,9), edgecolor='gray', facecolor='none', linewidth=1, alpha=0.9, scheme='Fisher_Jenks')

# Host 5 - 112
ax = sus5.plot(markersize=8, color = 'lightskyblue', scheme='Fisher_Jenks', ax=ax)
# Host 6 - 39
ax = sus6.plot(markersize=10, color = 'darkgreen', scheme='Fisher_Jenks', ax=ax)
# Host 7 - 23
ax = sus7.plot(markersize=10, color = 'darkorange', scheme='Fisher_Jenks', ax=ax)
# Host 8 - 15
ax = sus8.plot(markersize=10, color = 'darkred', scheme='Fisher_Jenks', ax=ax)

ax.set_title("Geographic Distribution of Sample Two\n\n4 hosts / 189 listings", fontsize=14)
plt.axis('off');

path = os.path.join('','img')
fn = 'fig7'

if not os.path.exists(path):
    print(f"Creating {path} under {os.getcwd()}")
    os.makedirs(path)
    
plt.savefig(os.path.join(path,fn), index=False)
print("Done.")

#### Coding Credits

1. [Yıldırım, S. (2021b). Data Cleaning and EDA on Airbnb Dataset with Python Pandas and Seaborn. Medium. Available at: https://towardsdatascience.com/data-cleaning-and-eda-on-airbnb-dataset-with-python-pandas-and-seaborn-7c276116b650 (Accessed: 25 January 2022).](https://towardsdatascience.com/data-cleaning-and-eda-on-airbnb-dataset-with-python-pandas-and-seaborn-7c276116b650)

2. [Yıldırım, S. (2021a). Airbnb Data Analysis With Pandas and Seaborn. Medium. Available at: https://betterprogramming.pub/airbnb-data-analysis-with-pandas-and-seaborn-11ceebba1eab (Accessed: 25 January 2022).](https://betterprogramming.pub/airbnb-data-analysis-with-pandas-and-seaborn-11ceebba1eab)

3. [Mendes, B. (2021). AirBnB_Data_Analysis_Toronto. Available at: https://github.com/brunacmendes/Airbnb_Data_Analysis_Toronto/blob/7970932ccaa473cf2fc8a9b6b0a698e733ac5b47/Airbnb_Listings_Data_Analysis_Toronto.ipynb (Accessed: 25 January 2022).](https://github.com/brunacmendes/Airbnb_Data_Analysis_Toronto/blob/main/Airbnb_Listings_Data_Analysis_Toronto.ipynb)

4. [‘End-to-End Predictive Analysis on AirBnB Listings Data’. (2021). Analytics Vidhya, 5 October. Available at: https://www.analyticsvidhya.com/blog/2021/10/end-to-end-predictive-analysis-on-airbnb-listings-data/ (Accessed: 25 January 2022).](https://www.analyticsvidhya.com/blog/2021/10/end-to-end-predictive-analysis-on-airbnb-listings-data/)

5. [Python/Project. AirBnB Part B (Python- Interactive Map, Natural Language Processing) .ipynb at master · Lukastuong123/Python. (no date). GitHub. Available at: https://github.com/Lukastuong123/Python (Accessed: 25 January 2022).](https://github.com/Lukastuong123/Python/blob/master/Project-%20Airbnb%20(Python-%20Interactive%20Map%2C%20Natural%20Language%20Processing%2C%20Comparative%20Study%2C%20Regression)/Project.%20AirBnB%20Part%20B%20(Python-%20Interactive%20Map%2C%20Natural%20Language%20Processing)%20.ipynb)

6. [Tukel, E. (2022). Spatial analysis of Istanbul Airbnb dataset using GeoPandas. Medium. Available at: https://towardsdev.com/spatial-analysis-of-istanbul-airbnb-dataset-using-geopandas-bc2403f8e2d8 (Accessed: 25 January 2022).](https://towardsdev.com/spatial-analysis-of-istanbul-airbnb-dataset-using-geopandas-bc2403f8e2d8)

7. [Week 5: Spatial Data Manipulation and Visualization I — Introduction to Geoprocessing. (no date). Available at: https://spatial-networks-lab.pages.gitlab.kuleuven.be/igp-course-website-2021/week5.html (Accessed: 25 January 2022).](https://spatial-networks-lab.pages.gitlab.kuleuven.be/igp-course-website-2021/week5.html)

8. [Nathan, P. (2020). Visualizing Geospatial Data in Python. Medium. Available at: https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621 (Accessed: 25 January 2022).](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621)

9. [Kumara, H. (2020). ‘How to drop rows based on column values using Pandas Dataframe’. Medium, 28 May. Available at: https://medium.com/@harsz89/how-to-drop-rows-based-on-column-values-using-pandas-dataframe-38cf50e4c95a (Accessed: 25 January 2022).](https://medium.com/@harsz89/how-to-drop-rows-based-on-column-values-using-pandas-dataframe-38cf50e4c95a)

10. [GeoPandas 0.10.2+0.g04d377f.dirty — GeoPandas 0.10.2+0.g04d377f.dirty documentation. (no date). Available at: https://geopandas.org/en/stable/index.html (Accessed: 25 January 2022).](https://geopandas.org/en/stable/index.html)

11. [Matplotlib documentation — Matplotlib 3.5.1 documentation. (no date). Available at: https://matplotlib.org/stable/index.html (Accessed: 25 January 2022).](https://matplotlib.org/stable/index.html)

12. [8 Python Pandas Value_counts() tricks that make your work more efficient. (2020). Re-thought. Available at: https://re-thought.com/pandas-value_counts/ (Accessed: 25 January 2022).](https://re-thought.com/pandas-value_counts/)

13. [Kumar, B. (2021). ‘Matplotlib Pie Chart Tutorial - Python Guides’, 26 December. Available at: https://pythonguides.com/matplotlib-pie-chart/ (Accessed: 25 January 2022).](https://pythonguides.com/matplotlib-pie-chart/)

---

## Part 2 - Executive Briefing

---

### 1 Executive Summary

As of 2021, online accommodation marketplace Airbnb had 5.6 million listings worldwide (Airbnb News 2021) enabling ordinary 
people to share ‘their homes a few nights a month to earn extra income’ (Ngabo 2018). However, the business model has given 
rise to a shadow industry characterized by the phenomenon of ghost hotels - short term rental (STRs) that are not occupied by 
the owner and operate are managed by a company (Chevalier 2019).

This shadow industry is negatively impacting the housing market when, in many cities globally, housing availability and 
affordability are in crisis (Shabrina et al 2021). This report provides an analysis of Airbnb activity in London based on data 
sourced from the Inside Airbnb website for the month of August 2019 – the last summer month prior to the COVID-19 pandemic.

Through exploratory data analysis we set out to answer three main questions about Airbnb in London:

1.	How many listings are not a primary residence of the host?
2.	To what extent are listings professionally managed?
3.	What is the geographic concentration of such listings?

#### Key Findings

- While only 18% of hosts have multiple listings, they represent 45.3% of all listings and an estimated 49% of all profits
- 115 hosts have 25+ listings each and account for 5952 listings alone 
- 11 boroughs have 1000+ listings by hosts with multiple listings
- Most multi-listing hosts seem to be well-established professional operations
- High concentrations of listings suggest not only ghost hotels, but ghost neighbourhoods

#### Recommendations to the Mayor

While implementing the ‘90-day rule’ in 2017, research suggests that further Airbnb regulations benefit cities with a higher 
tourism like London (Grimmer, et. al. 2019). Accordingly, the following recommendations aim to provide a path forward:

1.	Establish definitions for STRs and principal residences;
2.	Use the definitions to draft legislation requiring STRs to be registered and/or licensed with the city; and
3.	Use registration/licenses to enforce the following requirements:
    -	that STRs to include the registration/license number in their listing;
    -	that STRs only be permitted in an owner's principal residence; and
    -	that only STRs with registration/license numbers be permitted to be professionally managed.

---

### 2 Background

#### Airbnb

Airbnb is a global, online platform operating as a marketplace where users can ‘list their space and book unique accommodations 
anywhere in the world’ (Airbnb 2021). While Airbnb’s messaging suggests it facilitates homeowners to make extra income by 
offering their spare rooms or their entire home for when they are away (Airbnb 2021), there is also substantial evidence the 
platform both directly (Cox & Slee 2016) and indirectly (Bivens 2019) limits transparency needed for proper enforcement. On the 
surface, Airbnb’s promise as part of the sharing economy seems genuine, however, data has demonstrated the platform has 
transitioned to ‘commercial-style Airbnb’s’ (Nieuwland & van Melik 2020) where most listings are for ‘entire homes, many of 
which are rented all year round - disrupting housing and communities’ (Inside Airbnb 2022). 

#### Inside Airbnb

Inside Airbnb is an activist project which sources, cleans, analyzes and aggregates publicly available data from the Airbnb 
website. The service, which represents a growing movement pushing STR platforms to pay their fair share (Wachsmuth & Weisler 2018), 
scrapes data from the Airbnb website to contextualize impacts on housing markets and support ongoing debate (Inside Airbnb 2022). 
While owner information is not public, according to Fairbnb (2019) ‘Airbnb has access to data that makes it clear to them who those 
ghost hotel operators are’ and therefore could implement regulation to stop them.

#### 'Ghost Hotels'

Ghost hotels is a term used to describe Airbnb listings that are not part of the sharing economy, but rather ‘homes and units, 
often purchased in bulk by corporations and taken permanently off the rental market’ (ONeil 2019, Cocola-Gant and Gago 2019). 
By virtue of using an online platform and self check-in, they effectively operate as ‘small-scale lodging companies’ or ‘mini hotels’
(Bivens 2019) and are in complete opposition to the ‘connection [and] deeply personal experience’ (Chesky et al. 2021) that is, 
according to Airbnb, at its core. While some regulation has been put in place in London, such as automatic booking limits to 90 
nights per year (Airbnb 2022), ghost hotels have found workarounds to continue to operate and turn entire buildings into hotels (Temperton 2020).

---

### 3 Analysis

At all points of this analysis, fields that could potentially contain personal or identifiable information have been removed. This 
analysis has been completed based on the following methodology:

- **STEP 1:** Find ‘hotel rooms’ – filter listings based on minimum stay offered and by hosts with multiple listings
- **STEP 2:** Find ‘hotel staff’ – analyze similarity and professionalism within the name and descriptions fields of listings
- **STEP 3:** Find ‘hotels’ – explore the geographical concentration of listings by hosts with multiple listings

#### QUESTION ONE - How many listings are not the primary residence of the host?

Data from August 2019 revealed 84412 total listings. During data cleaning, only listings with minimum stays less than 28 days were kept. 
This left 76692 listings (98.3%), but only 51121 total hosts - a difference of 25571 listings. Next, all hosts with single-listings were 
removed, leaving only hosts with multiple listings for analysis – 34757 listings. 

Figure 1 provides a comparison of findings between single-hosts and multi-hosts. The findings of this report confirm that, like findings 
of prior studies conducted in other cities (Bivens 2019, Dogru, et. al. 2020), London’s Airbnb supply relies heavily on hosts of multiple 
units for its listings and revenue. It should be noted that People Accommodated and Profit are meant to be per night estimates assuming 
full bookings.

##### Figure 1 – Comparison of Single-host Listings to Multi-host Listings
![fig1](img/fig1.png)

#### QUESTION TWO - To what extent are listings professionally managed?

Two samples of four hosts each were taken to analyze their name and description fields. The first sample was from all multi-host listings. 
The second sample was from a sub-set of multi-host listings consisting only of private room listings. The hosts have been anonymized in this report.

##### Sample One

The four hosts sampled offered 753 listings, ranging from 646 to 25 listings each. Another sample of eight random listings was taken 
from each of the four hosts. The name and description fields used by three of the hosts were nearly identical and all three also 
classified their listings by a code – like a hotel suite. For example, one host had three different listings labelled GR R1, GR R4, 
and GR R7. While the host with 646 listings did not classify or use identical descriptions, the name of every listing included the name 
of a company. After further research, this company was determined to be a property management company that manages listings on behalf of 
private owners. It is more likely that this host is an agent than a homeowner and exemplifies how property management companies have come 
to represent the ‘professional business of Airbnb rentals’ (Leshinsky & Schatz 2018) while ensuring ‘absentee property owners’ (Bivens 2019) 
remain anonymous from enforcement.

##### Sample Two

The four hosts of sample two offered 189 listings, ranging from 112 to 15 listings each. Another sample of eight random listings was taken 
from each of these four hosts. Once again, the name and description fields of all host’s listings were either nearly identical or very similar. 
Similarly, three of the four hosts classified their listings by code. 

#### QUESTION THREE - What is the concentration of the listings identified above?

Figures 2 and 3 show the distribution of multi-host listings by borough. Multi-listings are spread throughout London but are highly 
concentrated in the inner-boroughs – especially Westminster, Tower Hamlets, Camden, and Kensington and Chelsea. This remains consistent 
with a 2016 study which found the highest concentration of Airbnb listings in the same four boroughs (Ferreri & Sanyal 2018).

##### Figure 2 - Density of Multi-host Listings by Borough
![fig4](image/fig4.png)

##### Figure 3 – Boroughs Ranked by Number of Multi-host Listings
![fig5](image/fig5.png)

Figure 4 shows the geographic distribution of sampled professionally managed listings. The green and blue hosts are geographically 
dispersed predominantly within the inner boroughs. The red and yellow hosts however are very concentrated. For example, the red host 
has 25 listings but seemingly only 3 overlapping points.

##### Figure 4 - Concentration of Sample One - Multi-host Listings
![fig6](image/fig6.png)

Figure 5 shows the geographic concentration of sampled private room listings. There is clearly a much higher concentration among this 
sample than the previous one. For instance, the red, yellow, and green hosts have 15, 23, and 39 listings each, but only appear to have 
4, 3, and 3 points each, respectively. There is a high probability that ghost hotels are present within this sample. The blue host seems 
more dispersed but is still highly centred on a single neighbourhood - Tower Hamlets. Collectively, all four hosts can be seen as an 
illustration of the potential for listings to dominate ‘entire apartment blocks or even neighbourhoods’ (Nieuwland & van Melik 2020). 

##### Figure 5 - Concentration of Sample Two - Private Room Listings
![fig7](image/fig7.png)

---

### 4 Conclusion

It is clear from this analysis that hosts with multiple listings dominate the STR market in London. The ’90-day-rule” has not prevented 
the displacement of rental units in favour of STR, income generating properties. Further rules need to be put in place starting by 
establishing clear definitions for STRs and principal residences. Once these definitions are established, they need to be linked through 
accompanying legislation that requires STRs to be registered/licensed by their owner and for the registration/license number to be displayed
in all listings of the STR.

Once STRs are required to be registered/licensed, enforcement becomes much easier. London authorities and administrative staff will be 
able to determine whether an STR is the owner’s private residence. Further, requiring STRs to be registered and/or licensed with the 
city would also allow enforcement to begin that prohibits professional management companies from representing or providing services to 
STRs that are not registered/licensed. By illuminating the practice, the negative impacts of the STR shadow industry can be monitored and addressed.

---

### 5 Bibliography

1. [Airbnb (2022) ‘About us’ (WWW) Airbnb Newsroom (https://news.airbnb.com/about-us/; 20 January 2022).](https://news.airbnb.com/about-us/)

2. [Airbnb (2022) ‘Responsible hosting in the United Kingdom’ (WWW) Airbnb Help Centre (https://www.airbnb.ca/help/article/1379/responsible-hosting-in-the-united-kingdom; 20 January 2022).](https://www.airbnb.ca/help/article/1379/responsible-hosting-in-the-united-kingdom)

3. [Airbnb (2022) ‘What is Airbnb and how does it work?’ (WWW) Airbnb Help Centre (https://www.airbnb.ca/help/article/2503/what-is-airbnb-and-how-does-it-work; 20 January 2022).](https://www.airbnb.ca/help/article/2503/what-is-airbnb-and-how-does-it-work)

4. [Bivens, J. (2019). ‘The economic costs and benefits of Airbnb: No reason for local policymakers to let Airbnb bypass tax or regulatory obligations’. Economic Policy Institute. Available at: https://www.epi.org/publication/the-economic-costs-and-benefits-of-airbnb-no-reason-for-local-policymakers-to-let-airbnb-bypass-tax-or-regulatory-obligations/ (Accessed: 26 January 2022).](https://www.epi.org/publication/the-economic-costs-and-benefits-of-airbnb-no-reason-for-local-policymakers-to-let-airbnb-bypass-tax-or-regulatory-obligations/)

5. [Chesky, B., Blecharcyzk, N. and Gebbia, J. (2021) ‘What Makes Airbnb, Airbnb’, (WWW) Airbnb Newsroom (https://news.airbnb.com/what-makes-airbnb-airbnb/; 20 January 2022).](https://news.airbnb.com/what-makes-airbnb-airbnb/)

6. [Chevalier, J. (2019) ‘These guys are predators’: Condo owner says home turned into Airbnb ‘ghost hotel’ (WWW) CBC News (https://www.cbc.ca/news/canada/ottawa/condo-owner-violated-airbnb-ghost-hotel-1.5122219; 20 January 2022).](https://www.cbc.ca/news/canada/ottawa/condo-owner-violated-airbnb-ghost-hotel-1.5122219)

7. [Cocola-Gant, A. and Gago, A. (2021) ‘Airbnb, buy-to-let investment and tourism-driven displacement: A case study in Lisbon’, Environment and Planning A: Economy and Space, 53(7), pp. 1671–1688.](https://doi.org/10.1177/0308518X19869012)

8. [Cox, M. and Slee, T. (2016). ‘How Airbnb’s data hid the facts in New York City’, p. 16.](http://insideairbnb.com/reports/how-airbnbs-data-hid-the-facts-in-new-york-city.pdf)

9. [Dogru, T., Mody, M., Suess, C., Line, N. and Bonn, M. (2020). ‘Airbnb 2.0: Is it a sharing economy platform or a lodging corporation?’ Tourism Management, 78, p. 104049. doi: 10.1016/j.tourman.2019.104049.](https://doi.org/10.1016/j.tourman.2019.104049)
        
10. [Ferreri, M. and Sanyal, R. (2018). ‘Platform economies and urban planning: Airbnb and regulated deregulation in London’. Urban Studies. SAGE Publications Ltd, 55 (15), pp. 3353–3368. doi: 10.1177/0042098017751982.](https://doi.org/10.1177/0042098017751982)
        
11. [Grimmer, L., Vorobjovas-Pinta, O. and Massey, M. (2019). ‘Regulating, then deregulating Airbnb: The unique case of Tasmania (Australia)’. Annals of Tourism Research, 75, pp. 304–307. doi: 10.1016/j.annals.2019.01.012.](https://doi.org/10.1016/j.annals.2019.01.012)
        
12. [Inside Airbnb (2022) ‘Inside Airbnb. Adding data to the debate’ (WWW) Inside Airbnb (http://insideairbnb.com; 20 January 2022).](http://insideairbnb.com/)

13. [Leshinsky, R. and Schatz, L. (2018). ‘“I Don’t Think My Landlord Will Find Out:” Airbnb and the Challenges of Enforcement’. Urban Policy and Research. Routledge, 36 (4), pp. 417–428. doi: 10.1080/08111146.2018.1429260.](https://doi-org.libproxy.ucl.ac.uk/10.1080/08111146.2018.1429260)

14. [Ngabo, G. (2018) ‘Home-sharing watchdog warns of “ghost hotels”’ (WWW) The Toronto Star (https://www.thestar.com/news/gta/2018/12/17/home-sharing-watchdog-warns-of-ghost-hotels.html; 20 January 2022).](https://www.thestar.com/news/gta/2018/12/17/home-sharing-watchdog-warns-of-ghost-hotels.html)

15. [Nieuwland, S. and van Melik, R. (2020). ‘Regulating Airbnb: how cities deal with perceived negative externalities of short-term rentals’. Current Issues in Tourism. Routledge, 23 (7), pp. 811–825. doi: 10.1080/13683500.2018.1504899.](https://doi-org.libproxy.ucl.ac.uk/10.1080/13683500.2018.1504899)

16. [O’Neil, L. (2019) ‘Toronto officially asks Airbnb to take down ghost hotels’ (WWW) BlogTO (https://www.blogto.com/real-estate-toronto/2019/02/airbnb-toronto-ghost-hotels-city-council-vote/; 20 January 2022).](https://www.blogto.com/real-estate-toronto/2019/02/airbnb-toronto-ghost-hotels-city-council-vote/)

17. [Shabrina, Z., Arcaute, E. and Batty, M. (2022a) ‘Airbnb and its potential impact on the London housing market’, Urban Studies, 59(1), pp. 197–221.](https://doi.org/10.1177/0042098020970865)

18. [Temperton, J. (2020) ‘I stumbled across a huge Airbnb scam that’s taking over London’ (WWW) Wired UK (https://www.wired.co.uk/article/airbnb-scam-london; 20 January2022).](https://www.wired.co.uk/article/airbnb-scam-london)

19. [Wachsmuth, D. and Weisler, A. (2018). ‘Airbnb and the rent gap: Gentrification through the sharing economy’. Environment and Planning A: Economy and Space. SAGE Publications Ltd, 50 (6), pp. 1147–1170. doi: 10.1177/0308518X18778038.](https://doi.org/10.1177/0308518X18778038)